In [1]:
import sys
sys.path.insert(0, "..")
from gpbp.layers import AdmArea
from gpbp import visualisation

from optimization import jg_opt
from functools import partial

## Defining the Administrative Area

In [ ]:
adm_area = AdmArea(country="Timor-Leste", level=1)

In [ ]:
adm_area.get_adm_area("Baucau")

## Retrieving Facility and Population data

In [ ]:
adm_area.get_facilities(method="osm", tags={"amenity":"hospital"})
visualisation.plot_facilities(adm_area.fac_gdf)

In [ ]:
adm_area.fac_gdf

In [ ]:
adm_area.get_population(method="world_pop")
visualisation.plot_population_heatmap(adm_area.pop_df)

## Computing potential locations for facilities

In [ ]:
adm_area.compute_potential_fac(spacing=0.05)
visualisation.plot_facilities(adm_area.pot_fac_gdf)

## Retrieving the road network

In [8]:
adm_area.get_road_network("driving")

## Prepare optimization data

In [9]:
MAPBOX_API_TOKEN = None # fill out with your own access token for mapbox strategy
DISTANCE_TYPE = "length"
pop_count, current, potential = adm_area.prepare_optimization_data(
    DISTANCE_TYPE, [2000, 5000, 10000], "driving", "osm", population_resolution=5, mapbox_access_token=MAPBOX_API_TOKEN)

## Optimize

PISA can perform optimizations using a variety of solvers. You can use a solver of your choice. For example, you can install the [COIN-OR Branch-and-Cut solver](https://github.com/coin-or/Cbc#download) and specify the path of the solver's binary below (on Linux/macOS you can find the path to the solver using `which cbc`).

In [ ]:
CBC_SOLVER_PATH = None # fill out the solver path where the cbc executable 
BUDGET = [5, 20, 50] # budget for the optimization in terms of how many locations can be built
cbc_optimize = partial(
                    jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH
                )
jg_opt.Solve(pop_count, current, potential, DISTANCE_TYPE, BUDGET, optimize=cbc_optimize, type='ID')